In [118]:
import numpy as np
import random
import pandas as pd
import copy

from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# World Sense Disambiguation (WSD)

Ce notebook permet d'effectuer une comparaison des performances de deux algorithmes, un d'apprentissage supervisé et un d'apprentissage semi-supervisé, pour la tâche de Word Sense Disambiguation (WSD). 

Les deux méthodes sont les suivants :
- pour la classification supervisée, utilisation d'un MLP
- pour la classification semi-supervisée, utilisation d'un algorithme de constrained K-Means

Plusieurs tests sont effectués en considérant plusieurs mots à désambiguiser pour lesquels les performances de ces deux méthodes sont évaluées. L'objectif final est de comparer ces performances pour déterminer à partir de combien de données annotées la première méthode est préférable à la deuxième et vice versa.



## Extraction des données

La classe Extractor permet d'accéder aux données et de les stocker dans des dictionnaires puis des embeddings. Elle permet de figer les données sur lesquelles les tests de classification seront ensuite faits. Elle comporte les méthodes suivantes :

- `extract_examples()` : pour extraire les données d'entraînement et de test provenant de fichiers XML FSE

- `extract_embeddings()` : pour extraire les embeddings à partir d'un fichier créé en amont, qui ne regroupe que les embeddings nécéssaires à notre jeu de données. Cette méthode est surtout utile pour drastiquement réduire le temps de chargement des embeddings

- `look_up()` : pour effectuer l'étape de look-up avant la classifcation

- `select_examples()` : pour sélectionner des données représentatives lorsqu'on ne considère pas toutes les données annotées. Pour qu'un ensemble de données soit représentatif, il faut que chaque étiquette présente dans les données totales soit présente au moins une fois dans le set d'entraînement sélectionné

- `define_instance()` : pour choisir les données qui seront classifiées, c'est-à-dire quelle proportion des données seront utilisées (80%, 50%, 25%...)

- `classifier_display_at_instance_time()` : pour afficher les données relatives au classifieur

- `k-means_display_at_instance_time()` : pour afficher les données relatives au constrained k-means

- `get_most_frequent_sense()` : pour obtenir le sens le plus fréquent et son occurence pour un lemme donné

In [119]:
class Extractor :
    
    def __init__(self, data_path, gold_path, embeddings_path, context_size):
        '''
        A partir des données, extrait les examples qui seront utilisés lors de l'apprentissage.

        INPUT
        data_path (str) : chemin pour accéder aux données d'entraînement, au format XML
        gold_path (str) : chemin pour accéder aux numéros de sens correspondant aux données
                          d'entraînement
        embeddings_path (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données
        context_size (int) : taille de la fenêtre du contexte du mot à désambiguiser

        ARGUMENTS DE CLASSE
        self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
                                 utilisation dans une phrase du corpus. 
                                 examples (list) : liste de tous les couples 
                                  ([mots du contexte], numéro de sens) pour un lemme donné
        self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        self.w2emb (dict) : associe chaque mot à son embedding
        self.selected_examples (liste) : liste d'examples qui contient au moins un example de 
                                         chaque sens du lemme choisi
        self.selected_examples_embs (liste) : même liste d'examples, mais avec les contextes 
                                              sous forme d'embedding
        self.X_train (list) : corpus d'apprentissage
        self.X_test (list) : numéros des sens correspondant au corpus d'apprentissage
        self.y_train (list) : corpus de test
        self.y_test (list) : numéros des sens correspondant au corpus d'apprentissage
        self.annotated_examples (list) : liste des données considérées comme annotées 
        self.unannotated_examples (list) : liste des données considérées comme non annotées 
        '''
        
        #récupération des données XML d'apprentissage fournies en argument
        tree = ET.parse(data_path)
        data_file = tree.getroot()

        #récupération des données txt correspondant aux gold class
        gold_file = open(gold_path, "r",encoding="utf-8")
        
        # self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
        #                          utilisation dans une phrase du corpus.
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        #
        # self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        self.w2examples, self.w2senses = self.extract_examples_and_senses(data_file, gold_file, context_size)

        # self.w2emb (dict) : associe chaque mot à l'embedding représentant son contexte sommé
        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding
        self.w2emb = self.extract_embeddings(embeddings_path)

        # liste d'examples qui contient au moins un example de chaque sens du lemme choisi
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples = []
        # même liste d'examples, mais avec les contextes sous forme d'embedding
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples_embs = []
        # liste des examples non selectionnés, dont les contextes sont sous forme d'embedding
        self.unannotated_set = []

        # les corpus seront définis lorsque l'instance du mot à désambiguiser sera choisi
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []

        # liste des données considérées comme annotées 
        self.annotated_examples = []
        # liste des données considérées comme non annotées 
        self.unannotated_examples = []
    
    def extract_examples_and_senses(self, data_file, gold_file, context_size):
        '''
        Extrait les données à partir des fichiers de corpus d'apprentissage et de gold classes.

        INPUT
        data_file (Element): représentation des phrases du corpus d'apprentissage
        gold_file (TextIOWrapper): fichier contenant les numéros de sens (gold class) pour chaque
                                   mot à désambiguiser
        '''
    
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        w2examples = {}
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        w2senses = defaultdict(set)

        # lecture du fichier gold
        gold_file = gold_file.readlines()
        
        # index de parcours dans le fichier gold
        i_gold = 0

        # pour chaque phrase du corpus
        for text_id in data_file:
            for sentence in text_id:

                # recherche de(s) l'indice(s) de(s) l'instance(s) pour savoir où se trouve la fenêtre
                i_instances = []
                for j in range(len(sentence)):
                    if sentence[j].tag == "instance":
                        i_instances.append(j)
                
                # tant qu'il y a des instances à repérer dans la phrase
                while len(i_instances) > 0:

                    # enregistrement du lemme de l'instance
                    instance = sentence[i_instances[0]].attrib["lemma"].lower()
                    
                    # si l'instance n'a pas encore de contexte
                    if instance not in w2examples : 
                        # le crée
                        w2examples[instance] = []

                    context = []
                    # pour chaque mot de la fenêtre
                    for k in range(-context_size, context_size + 1):
                        # si le mot est l'instance, enregistrement de son lemme
                        if k == 0:
                            context.append(instance)
                        # sinon enregistrement du mot du contexte
                        elif len(sentence) > i_instances[0] + k and i_instances[0] + k >= 0:
                            context.append(sentence[i_instances[0] + k].text.lower())
                    
                    # récupération des différents sens possibles dans le fichier gold 
                    gold_class = gold_file[i_gold].split("__")[1].split("_")[1]

                    # le fichiers gold et data ayant les données dans le même ordre, les instances
                    # et les sens peuvent être enregistrés en même temps dans leur dictionnaire
                    # respectif
                    w2senses[instance].add(gold_class)
                    w2examples[instance].append((context, gold_class))
                    
                    # l'instance lue est enlevée de la to-do list et l'index de parcours du fichier 
                    # gold est incrémenté pour passer à l'instance suivante
                    i_instances.pop(0)
                    i_gold += 1

        return w2examples, w2senses
    
    def extract_embeddings(self,path_embeddings) :
        '''
        Récupère les embeddings dans le fichier généré et associe à chaque mot son
        embedding.

        INPUT
        path_embeddings (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données

        OUTPUT
        w2emb (dict) : associe chaque mot à son embedding
        '''

        # lecture du fichier
        file_embeddings = open(path_embeddings , "r", encoding="UTF-8")

        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding du mot
        w2emb = {}

        # pour chaque mot du fichier
        for line in file_embeddings.readlines():

            # séparation du mot et de l'embedding
            splitted_line = line.split(" ")
            word = splitted_line[0]
            embedding = list(map(float,splitted_line[1:]))
            # insertion du mot et de son embedding dans le dictionnaire
            w2emb[word] = embedding

        return w2emb

    def look_up(self, context) :
        '''
        Remplace dans les mots du vecteur de contexte par leur embedding et en fait la
        somme.

        INPUT
        context (list) : liste de taille (size_window*2)+1

        OUTPUT
        context_emb (list) : liste de taille size_embedding (300 ici)
        '''

        # taille d'un embedding : 300
        emb_size = len(list(self.w2emb.values())[0]) 
        # initialisation du contexte sous forme d'embedding
        context_emb = np.zeros(emb_size)

        # pour chaque mot du contexte
        for word in context :
            # s'il est présent dans le dictionnaire d'embeddings
            if word in self.w2emb :
                # il est ajouté à l'embedding représentant le contexte
                context_emb = np.add(context_emb, np.array(self.w2emb[word])) 

        return context_emb
    
    def select_examples(self, examples, senses, sample_size):
        '''
        Choisit des examples représentatifs du corpus selon un nombre imposé de 
        données à choisir.

        INPUT
        examples (list) : liste d'examples pour le mot à désambiguiser
        n_senses (int) : nombre de sens associés à ce mot
        sample_size (float) : quantité des données considérés

        OUTPUT
        selected_examples (list) : liste d'examples qui contient au moins un example 
                                   de chaque sens
        '''

        selected_examples = []
        
        # pour chaque sens, on ajoute un example associé à ce sens, au hasard
        for sense in senses :
            selected_examples.append(random.choice(list(filter((lambda example:example[1]==sense),
                                                               examples))))
        
        # ajoute le nombre de données manquantes (non-présentes dans la liste) selectionnées au hasard
        random.shuffle(examples)
        for example in examples:
            if example not in selected_examples and len(selected_examples) < sample_size*len(examples):
                selected_examples.append(example)

        return selected_examples
    
    def select_train_test(self, x, y, senses, data_size, display, train_size = 0.8):
        '''
        Sépare en un train set et test set les examples représentatifs du corpus en mettant 
        au moins une étiquette de chaque sens dans le train set (et le test set si possible)

        INPUT
        x (list) : liste d'examples pour le mot à désambiguiser
        y (list) : liste d'étiquette pour chaque example de x
        senses (list) : numéros des sens associés à ce mot
        train_size (float) : quantité des données d'entraînement considérés
        '''

        # SELECTION DES DONNEES REPRESENTATIVES DU TRAIN

        # sélection au hasard d'un contexte et de son étiquette pour chaque sens
        selected_train = [random.choice(list(filter((lambda example: example[1] == sense),
                                                    zip(x,y)))) for sense in senses]
            
        # SELECTION DES DONNEES REPRESENTATIVES DU TEST

        # fait de même pour le corpus de test, lorsque possible, en ne considérant pas
        # ceux déjà sélectionnés pour le train
        selected_test = []
        for sense in senses:
            added = False
            for example in zip(x,y):
                for train_ex in selected_train:
                    # vérifie que l'example n'a pas déjà été sélectionné dans le train et
                    # le test pour ne pas faire de doublon
                    if example[1] == sense and example[1] == train_ex[1] and not(np.array_equal(example[0], train_ex[0])):
                        selected_test.append(example)
                        added = True
                        break
                if added == True:
                    break

        # SELECTION DES DONNEES RESTANTES DU TRAIN ET DU TEST

        # effectue un split sur le reste d'examples disponibles
        remaining_X = []
        remaining_y = []
        selected_train_copy = copy.deepcopy(selected_train)
        selected_train_copy.extend(selected_test)
        for contexte, sens in zip(x,y):
            to_add = True
            for ex in selected_train_copy:
                # vérifie que l'example n'a pas déjà été sélectionné dans le train et le
                # test pour ne pas faire de doublon
                if np.array_equal(contexte, ex[0]) and sens == ex[1]:
                    to_add = False
                    break
            if to_add:
                remaining_X.append(contexte)
                remaining_y.append(sens)

        try:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(remaining_X, remaining_y,train_size = train_size)
    
            # ajout des données représentatives dans les sets correspondant
            selected_X_train, selected_y_train = zip(*selected_train)
            selected_X_test, selected_y_test = zip(*selected_test)
            
            self.X_train.extend(selected_X_train)
            self.X_test.extend(selected_X_test)
            self.y_train.extend(selected_y_train)
            self.y_test.extend(selected_y_test)
            
        except:
            if display:
                print(f"/!\ ERREUR /!\ La taille des données est trop faible. L'opération train_test_split() est impossible.")                                                                     
            self.X_train, self.X_test, self.y_train, self.y_test = set(), set(), set(), set()
                     
    def define_instance(self, instance, data_size, display):
        '''
        Permet de définir l'instance du mot à désambiguiser et les examples à utiliser. 
        Fixe les derniers arguments de classe selon l'instance choisie.
        Contraint le choix de classification selon le nombre de données considéré.

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float): quantité de données à considérer
        display (bool, optional): affiche une trace des options choisies
        '''

        # SET DE DONNEES ANNOTEES

        # choisit des examples d'entraînement représentatifs du corpus selon le nombre
        # imposé de données
        self.annotated_examples = self.select_examples(self.w2examples[instance],
                                                       self.w2senses[instance],
                                                       data_size)
        # liste de tous les contextes sous forme d'embedding
        X_annotated = [self.look_up(context) for context, gold in self.annotated_examples]
        # liste de tous les numéros de sens
        y_annotated = [gold for context, gold in self.annotated_examples]
        # le set des données annotées est une liste des examples représentatifs sous forme d'embeddings
        self.selected_examples_embs = list(zip(X_annotated,y_annotated))

        # SET DE DONNEES NON ANNOTEES

        # garde en mémoire les examples considérés comme non-annotés
        self.unannotated_examples = [example for example in self.w2examples[instance] 
                                     if example not in self.annotated_examples]        
        # liste de tous les contextes sous forme d'embedding
        X_unannotated = [self.look_up(context) for context,gold in self.unannotated_examples]
        # liste de tous les numéros de sens
        y_unannotated = [gold for context,gold in self.unannotated_examples]
        # le set des données non-annotées est une liste des examples sous forme d'embeddings
        self.unannotated_set = list(zip(X_unannotated,y_unannotated))

        # SET DE DONNEES A CLASSIFIER PAR MLPCLASSIFIER_WSD
        
        # création des corpus d'entrainement et de test
        self.select_train_test(X_annotated, y_annotated, self.w2senses[instance],display,data_size)
            
        # affichage des informations relatives à l'instance choisie pour MLPClassifier_WSD
        if display :
            self.classifier_display_at_instance_time(instance, data_size)
            
    
    def classifier_display_at_instance_time(self, instance, data_size):
        '''
        Affiche les informations relatives à l'instance choisie pour MLPClassifier_WSD

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float) : quantité de données à considérer
        '''
        
        print("--> Instance :", instance)
        print(f'{data_size*100}% des données annotées considérées')
        print("Nombre de données d'entraînement : ", len(self.X_train))
        print("Etiquettes possibles pour cette instance : ", self.w2senses[instance])
        if self.y_train:
            print("Etiquettes présentes dans les données d'entraînement :", Counter(self.y_train))
            

    def kmeans_display_at_instance_time(self, instance, size):
        ''' 
        Affiche des informations relatives à l'instance choisie pour K_Means

        INPUT
        instance (str) : mot à désambiguiser
        num_correct (int) : nombre de prédictions correctes
        num_examples (int) : nombre d'examples total
        size (float) : quantité de données à considérer 
        '''
        y = [gold for tensor,gold in self.selected_examples_embs] + [gold for tensor,gold in self.unannotated_set]
        print("--> Instance :",instance)
        print(f"Proportion de données annotées considérées sur le corpus: {size*100} %")
        most_frequent_sense = max(y,key=y.count)
        occurrence_of_most_frequent_sense = y.count(most_frequent_sense)/len(y)
        print(f"Répartition des sens: {Counter(y)}")
        print(f"Le sens le plus fréquent pour '{instance}' est le sens {most_frequent_sense} avec une proportion de {round(occurrence_of_most_frequent_sense*100,2)} %")
        #print(f"Pour '{instance}', l'accuracy est de {round(num_correct/num_examples,2)} %")
        print()


    def get_sets(self):
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def get_annotated_and_unannotated_sets(self):
        return self.selected_examples_embs, self.unannotated_set
        
    def get_annotated_examples(self):
        return self.annotated_examples
    
    def get_unannotated_examples(self):
        return self.unannotated_examples
    
    def get_embs(self):
        return self.selected_examples_embs

In [120]:
# chemin du corpus annoté
data_path = "../donnees/FSE-1.1-191210/FSE-1.1.data.xml"
# chemin pour récupérer les gold class du corpus annoté
gold_path = "../donnees/FSE-1.1-191210/FSE-1.1.gold.key.txt"
# choix de la fenêtre du contexte
context_size = 4
# chemin pour récupérer les embeddings afin de faire l'opération look-up
# Les embeddings sont extraits de fasttext
embeddings_path = "../donnees/embeddings.txt"
# choix de l'instance à tester pour la classification
instance = "aboutir"

In [121]:
# exemple d'utilisation d'Extractor avec le verbe "aboutir" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance(instance, 0.8, display=True)
print("\nexemple d'example avec contexte et gold class :",ext.get_annotated_examples()[0])
print("\nau format embedding : ",ext.get_embs()[0])

--> Instance : aboutir
80.0% des données annotées considérées
Nombre de données d'entraînement :  31
Etiquettes possibles pour cette instance :  {'1', '4', '2', '3'}
Etiquettes présentes dans les données d'entraînement : Counter({'3': 25, '4': 4, '1': 1, '2': 1})

exemple d'example avec contexte et gold class : (['quai', 'de', 'grenelle', 'et', 'aboutir', 'rue', 'de', 'la', 'croix'], '1')

au format embedding :  (array([ 1.1260e-01,  1.1970e-01,  6.5350e-01, -6.0890e-01, -1.0010e-01,
       -1.3570e-01,  7.1280e-01, -2.0900e-02, -2.0390e-01, -6.1000e-01,
       -1.9460e-01, -3.1340e-01,  1.1800e-01,  5.8740e-01, -8.3500e-02,
        1.8990e-01,  5.5980e-01, -2.1220e-01, -4.5650e-01,  1.0880e-01,
        1.7000e-03, -4.7080e-01, -4.1750e-01,  3.7060e-01,  1.4750e-01,
        1.6180e-01,  1.1130e-01, -2.0490e-01,  3.9200e-01,  2.4550e-01,
        1.0200e-01, -2.3870e-01,  1.4240e-01, -2.2780e-01,  3.0380e-01,
        3.9980e-01, -7.0890e-01,  3.6210e-01,  3.1320e-01,  8.0000e-02,
       


## Apprentissage supervisé

L'utilisation d'un Classifieur MLP permet de prédire le sens des verbes de façon supervisée. Il s'agit de la première méthode proposée pour la tâche de WSD.

Les données utilisées sont les données annotées extraites du fichier FSE grâce à l'Extractor pour un lemme donné. Un MLPClassifier_WSD doit donc être créé pour chaque verbe dont on veut la prédiction.

MLPClassifier_WSD comporte les méthodes suivantes :
- `classify()` : pour prédire le sens du lemme associé au classifieur et calculer l'accuracy de cette prédiction
- `get_accuracy_baseline()` : pour récupérer le sens le plus fréquent et son nombre d'occurences dans les données d'entraînement

In [122]:
class MLPClassifier_WSD :
    
    def __init__(self, X_train, X_test, y_train, y_test) :
        '''
        A partir des données, prédit le sens d'un lemme et évalue la prédiction

        ARGUMENTS DE CLASSE
        self.X_train (list) : liste des contextes d'entraînement
        self.y_train (list) : liste des gold class d'entraînement
        self.X_test (list) : liste des contextes de test
        self.y_test (list) : liste des gold class de test
        self.accuracy (float) : accuracy du classifieur
        self.accuracy_baseline (float) : accuracy  du classifieur selon la baseline

        '''
        # ensembles d'entraînement et de test provenant de l'Extractor pour un lemme donné
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test 

        # prédiction du modèle
        self.y_pred = []
        # accuracy de la prédiction 
        self.accuracy = -1
        #accuracy de la baseline :most frequent sense
        self.accuracy_baseline = -1
        #most frequent sens
        self.mfs = -1
        
            
    def classify(self, display=False) :
        ''' 
        Permet de prédire le sens du verbe.

        INPUT
        display (bool, optional) : affiche une trace des options choisies
        '''
        try:
            clf = MLPClassifier(random_state=1, hidden_layer_sizes=(100,)) 
            # entraînement
            clf.fit(self.X_train, self.y_train)
            # prédiction
            self.y_pred = clf.predict(self.X_test)

            # calcul de l'accuracy
            self.accuracy = accuracy_score(self.y_pred, self.y_test)
            
            #calcul de l'accuracy de la baseline et récupération du most frequent sense
            self.accuracy_baseline, self.mfs = self.get_accuracy_baseline()
        
            # si une trace est souhaitée
            if display :
                print("prediction :", self.y_pred)
                print("gold :", self.y_test)
                print("accuracy score : ", round(self.accuracy,2))
                print("accuracy score de la baseline :",self.accuracy_baseline)
                
        except:
             print("/!\ ERREUR /!\: La quantité de données fournies est insuffisante: formation du test set impossible")
        
    def get_accuracy_baseline(self):
        ''' 
        Retourne le nombre d'étiquettes correspondantes au sens le plus fréquent et le sens le plus fréquent.

        OUTPOUT
        accuracy_baseline (float) : l'accuracy correspondante à la baseline
        mfs (int) : sens le plus fréquent
        '''
        
        #si la quantité de donées sélectionnées est suffisante
        if self.y_train :
            count_gold = Counter(self.y_train)
            #accuracy de la baseline : on divise le nombre d'étiquette 
            # dont le gold correspond au sens le plus fréquent par le nombre d'étiquettes prédites
            accuracy_baseline = round(max(count_gold.items(),key = lambda item:item[1])[1]/len(self.y_train),2)
            mfs = max(count_gold.items(),key = lambda item:item[1])[0]
        else :
            mfs = -1
            accuracy_baseline = -1.0
        return accuracy_baseline, mfs

In [123]:
# exemple d'utilisation de MLPClassifieur_WSD avec le verbe "demeurer" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("demeurer", 0.1, display=True)
X_train, X_test, y_train, y_test = ext.get_sets()

Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
Clf.classify(display=True)

/!\ ERREUR /!\ La taille des données est trop faible. L'opération train_test_split() est impossible.
--> Instance : demeurer
10.0% des données annotées considérées
Nombre de données d'entraînement :  0
Etiquettes possibles pour cette instance :  {'4', '3', '6', '2'}
/!\ ERREUR /!\: La quantité de données fournies est insuffisante: formation du test set impossible


Pour pouvoir comparer les performances de ce Classifieur MLP avec celles de l'apprentissage semi-supervisé, nous devons créer un classifieur par verbe à désambiguiser et choisir un pas de descente pour la quantité de données annotées considérées.

Pour chaque classifieur et chaque quantitée de données considérées, on effectue *n_repeat* classifications pour obtenir une accuracy moyenne ainsi qu'un nombre d'étiquette correctement prédit moyen dans un souci de représentativité.

Par conséquent, pour :
- n_repeat = 5
- step = 0.25

nous obtiendrons pour chaque classifieur une liste d'accuracies correspondant à :
- la moyenne des accuracies de 5 prédictions 
- la moyenne du nombre d'étiquettes correctement prédites de 5 prédictions 
- pour 100%, 75%, 50% et 25% des données.

S'ajoute à cela le calcul la macro-average de l'accuracy qui s'effectue grâce aux listes obtenues ainsi que celui de la micro-average accuracy.

Pour avoir une base de comparaison autonome pour chaque méthode, nous effectuons les mêmes évaluations en considérant des prédications selon la baseline du most frequent sense.

In [124]:
def get_mean_accuracies(instance, data_size, n_repeat, display=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné
    ainsi qu'une moyenne du nombre d'étiquette correctement prédites ainsi que le nombre
    d'étiquettes prédites, nécéssaires au calcul de la micro-average.

    INPUT
    n_repeat (int): nombre de classifications à effectuer
    display (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean_acc (float): moyenne des accuracies calculées
    mean_acc_baseline (float) : moyenne des accuracies de la baseline
    mean_corr_pred (int) : nombre d'étiquettes correctement prédites selon le classifieur
    mean_corr_pred_baseline (int) : nombre d'étiquettes correctement prédites selon la baseline
    n_pred (int) : nombre d'étiquettes prédites
    '''

    # liste de l'accuracy à chaque répétition
    accuracies = []
    
    # liste de l'accuracy de la baseline à chaque répétition
    accuracies_baseline = []
    
    #liste du nombre d'étiquettes correctement prédites
    n_corr_pred = []
    
    #liste du nombre d'étiquettes correctement prédites selon la baseline
    n_corr_pred_baseline = []
    
    #nombre d'étiquettes prédites
    n_pred = 0
    
    for n in range(n_repeat) :
            
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données
        ext.define_instance(instance, data_size, False)
        X_train, X_test, y_train, y_test = ext.get_sets()
        
        # si les sets ne sont pas vides i.e que la quantité de données sélectionnées est suffisante
        if y_train :
            
            # les données utiles à l'évaluation sont récupérées du classifieur
            Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
            Clf.classify()
            accuracies.append(Clf.accuracy)
            accuracies_baseline.append(Clf.accuracy_baseline)
            n_pred = len(y_test)
            n_corr_pred.append(sum([1 for (gold,pred) in zip(y_test,Clf.y_pred) if gold==pred]))
            n_corr_pred_baseline.append(sum([1 for gold in y_test if Clf.mfs==gold]))
            
        # lorsque la quantité de données sélectionnées est trop faible
        else :
            accuracies.append(-1.0)
            accuracies_baseline.append(-1.0)
            n_corr_pred.append(-1)
            n_corr_pred_baseline.append(-1)
    
    # calcul des moyennes 
    mean_acc = sum(accuracies)/len(accuracies)
    mean_acc_baseline = sum(accuracies_baseline)/len(accuracies_baseline)
    mean_corr_pred = sum(n_corr_pred)/len(n_corr_pred)
    mean_corr_pred_baseline = sum(n_corr_pred_baseline)/len(n_corr_pred_baseline)
            
    # si la trace est souhaitée
    if display :
        print(f'{data_size*100}% des données annotées considérées')
        print("instance :", instance)
        print("accuracies :", accuracies)
        print("accuracy moyenne :", mean_acc)
        print("accuracies de la baseline :",accuracies_baseline)
        print("accuracy moyenne de la baseline :", mean_acc_baseline,"\n")

    return round(mean_acc,2), round(mean_acc_baseline,2), round(mean_corr_pred,2), round(mean_corr_pred_baseline,2), round(n_pred,2)

def get_accuracies(instances, step, n_repeat, display=True):
    '''
    - Permet d'obtenir des accuracies moyennes par lemme pour une certaine quantité 
      de données considérée.
    - Produit des fichiers csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes + macro_average
    instance2acc_base (dict) : associe à chaque instance sa liste de baselines d'accuracies + macro_average
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    instance2acc = {instance : [] for instance in instances}
    instance2acc_base = {instance : [] for instance in instances}
    
    # listes des métriques d'évaluation calculées pour chaque quantité de données
    macro_average_acc = []
    macro_average_acc_base = []
    micro_average = []
    micro_average_baseline = []
    
    # compteurs pour le nombre d'étiquettes correctement prédites selon le classifieur et selon la baseline
    n_corr_pred = 0
    n_corr_pred_baseline = 0
    
    # nombre d'étiquettes prédites pour la quantité de données entrée
    n_pred_tot = 0
    
    # liste qui permettra d'afficher la quantité de données considérées dans le .csv
    data_sizes =[]
            
    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (100%, 75%, 50%, 25%)
        data_size = 1.0 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de ses accuracies moyennes, selon le classifieur et selon la baseline
            # comptage des étiquettes correctement prédites et du nombre d'étiquettes prédites
            mean_acc, mean_acc_baseline, mean_corr_pred, mean_corr_pred_baseline, n_pred = get_mean_accuracies(instance, data_size, n_repeat, display)
            
            instance2acc[instance].append(mean_acc)
            instance2acc_base[instance].append(mean_acc_baseline)
            n_corr_pred += mean_corr_pred
            n_corr_pred_baseline += mean_corr_pred_baseline
            n_pred_tot += n_pred
        
        # on boucle sur les valeurs obtenues pour obtenir leur macro-average et leur micro_average, selon le classifieur et selon la baseline
        accuracies = [accuracies[i] for accuracies in instance2acc.values() if accuracies[i] != -1.0]
        macro_average_acc.append(sum(accuracies)/len(accuracies))
        
        accuracies_base = [accuracies_baseline[i] for accuracies_baseline in instance2acc_base.values() if accuracies_baseline[i] != -1.0]
        macro_average_acc_base.append(sum(accuracies_base)/len(accuracies_base))
        
        micro_average.append(n_corr_pred/n_pred_tot)
        micro_average_baseline.append(n_corr_pred_baseline/n_pred_tot)
  
    # ajout d'une clé macro_average pour avoir les moyennes de toutes les accuracies moyennes pour chaque instance
    # pour chaque pas
    instance2acc["macro_average"] = macro_average_acc
    instance2acc_base["macro_average"] = macro_average_acc_base
    
    # ajout d'une clé micro_average pour avoir les micro_averages pour chaque instance
    # pour chaque pas
    instance2acc["micro_average"] = micro_average
    instance2acc_base["micro_average"] = micro_average_baseline
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    instance2acc["data_sizes"] = data_sizes
    instance2acc_base["data_sizes"] = data_sizes 

    # export en csv des résultats
    df1 = pd.DataFrame(instance2acc)
    df1.set_index("data_sizes")
    df1.to_csv(f"results/classifieur_accuracies_par_lemme_{n_repeat}_repet.csv")
    
    df2 = pd.DataFrame(instance2acc_base)
    df2.set_index("data_sizes")
    df2.to_csv(f"results/classifieur_accuracies_baseline_par_lemme_{n_repeat}_repet.csv")
    
    
    return instance2acc,instance2acc_base


In [125]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une precision moyenne
n_repeat = 5

In [126]:
# récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())
# calcul des precisions
get_accuracies(instances, step, n_repeat)

100.0% des données annotées considérées
instance : aboutir
accuracies : [0.9090909090909091, 0.9090909090909091, 0.6363636363636364, 0.8181818181818182, 0.9090909090909091]
accuracy moyenne : 0.8363636363636363
accuracies de la baseline : [0.82, 0.82, 0.87, 0.82, 0.82]
accuracy moyenne de la baseline : 0.8299999999999998 

100.0% des données annotées considérées
instance : investir
accuracies : [0.75, 0.75, 0.5833333333333334, 0.5833333333333334, 0.3333333333333333]
accuracy moyenne : 0.6000000000000001
accuracies de la baseline : [0.66, 0.66, 0.71, 0.71, 0.79]
accuracy moyenne de la baseline : 0.7060000000000001 

100.0% des données annotées considérées
instance : traduire
accuracies : [0.75, 0.8333333333333334, 0.8333333333333334, 0.6666666666666666, 0.75]
accuracy moyenne : 0.7666666666666667
accuracies de la baseline : [0.5, 0.55, 0.5, 0.55, 0.47]
accuracy moyenne de la baseline : 0.514 

100.0% des données annotées considérées
instance : témoigner
accuracies : [0.9166666666666666,

c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2

100.0% des données annotées considérées
instance : justifier
accuracies : [0.9090909090909091, 0.8181818181818182, 0.7272727272727273, 0.9090909090909091, 0.8181818181818182]
accuracy moyenne : 0.8363636363636363
accuracies de la baseline : [0.87, 0.9, 0.87, 0.87, 0.87]
accuracy moyenne de la baseline : 0.876 

100.0% des données annotées considérées
instance : viser
accuracies : [1.0, 0.9166666666666666, 0.9166666666666666, 1.0, 1.0]
accuracy moyenne : 0.9666666666666666
accuracies de la baseline : [0.58, 0.55, 0.58, 0.58, 0.53]
accuracy moyenne de la baseline : 0.5640000000000001 



c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


100.0% des données annotées considérées
instance : prononcer
accuracies : [0.5384615384615384, 0.46153846153846156, 0.38461538461538464, 0.6923076923076923, 0.38461538461538464]
accuracy moyenne : 0.4923076923076922
accuracies de la baseline : [0.41, 0.35, 0.38, 0.38, 0.3]
accuracy moyenne de la baseline : 0.364 

100.0% des données annotées considérées
instance : accomplir
accuracies : [0.6153846153846154, 0.7692307692307693, 0.6923076923076923, 0.5384615384615384, 0.6153846153846154]
accuracy moyenne : 0.6461538461538461
accuracies de la baseline : [0.84, 0.81, 0.81, 0.89, 0.86]
accuracy moyenne de la baseline : 0.842 



...

50.0% des données annotées considérées
instance : appartenir
accuracies : [0.5714285714285714, 0.2857142857142857, 0.3333333333333333, 0.42857142857142855, 0.5]
accuracy moyenne : 0.4238095238095238
accuracies de la baseline : [0.61, 0.61, 0.47, 0.56, 0.53]
accuracy moyenne de la baseline : 0.556 



c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


50.0% des données annotées considérées
instance : arriver
accuracies : [0.25, 0.25, 0.42857142857142855, 0.375, 0.5]
accuracy moyenne : 0.3607142857142857
accuracies de la baseline : [0.6, 0.45, 0.52, 0.5, 0.5]
accuracy moyenne de la baseline : 0.514 

50.0% des données annotées considérées
instance : chercher
accuracies : [0.5714285714285714, 0.5714285714285714, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571]
accuracy moyenne : 0.7428571428571429
accuracies de la baseline : [0.72, 0.56, 0.56, 0.44, 0.5]
accuracy moyenne de la baseline : 0.556 

50.0% des données annotées considérées
instance : composer
accuracies : [1.0, 0.5, 1.0, 1.0, 1.0]
accuracy moyenne : 0.9
accuracies de la baseline : [0.8, 0.85, 0.75, 0.75, 0.75]
accuracy moyenne de la baseline : 0.78 

50.0% des données annotées considérées
instance : confier
accuracies : [1.0, 0.8571428571428571, 0.7142857142857143, 0.8571428571428571, 0.8571428571428571]
accuracy moyenne : 0.8571428571428571
accuracies de la base

c:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


({'aboutir': [0.84, 0.84, 0.77, 0.68],
  'investir': [0.6, 0.48, 0.57, 0.35],
  'traduire': [0.77, 0.76, 0.71, 0.63],
  'témoigner': [0.82, 0.84, 0.77, 0.7],
  'juger': [0.72, 0.67, 0.71, 0.73],
  'justifier': [0.84, 0.8, 0.87, 0.73],
  'viser': [0.97, 0.89, 0.97, 0.85],
  'prononcer': [0.49, 0.39, 0.38, 0.45],
  'accomplir': [0.65, 0.59, 0.68, 0.65],
  'convenir': [0.62, 0.53, 0.42, 0.45],
  'acquérir': [0.59, 0.55, 0.58, 0.45],
  'achever': [0.89, 0.8, 0.73, 0.65],
  'observer': [0.49, 0.48, 0.52, 0.55],
  'adapter': [0.52, 0.51, 0.46, 0.63],
  'admettre': [0.7, 0.71, 0.74, 0.85],
  'entraîner': [0.8, 0.82, 0.7, 0.85],
  'payer': [0.51, 0.53, 0.35, 0.25],
  'respecter': [0.78, 0.67, 0.8, 0.85],
  'affecter': [0.62, 0.53, 0.55, 0.47],
  'demeurer': [0.69, 0.78, 0.68, 0.8],
  'aggraver': [0.85, 0.8, 0.66, 0.7],
  'agir': [0.66, 0.63, 0.55, 0.47],
  'ajouter': [0.58, 0.56, 0.52, 0.61],
  'alimenter': [0.71, 0.67, 0.67, 0.65],
  'coûter': [0.43, 0.4, 0.57, 0.32],
  'relancer': [0.35, 0.4

## Apprentissage semi-supervisé

Le constrained k-means est la deuxième méthode proposée pour la tâche WSD. Les données utilisées sont les mêmes que celles utilisées le Classifieur MLP.


In [127]:
class Constrained_K_Means():
    ''' 
    Classifieur K-means pour un mot particulier
    '''

    def __init__(self, annotated_examples, not_annotated_examples):
        '''
        Instancie les différentes variables utiles pour l'algorithme du K-means

        INPUT
        annotated_examples (list) : liste des exemples qui seront considérés
                                     comme annotés pour la construction des premiers
                                     centroïdes
        non_annotated_examples (list) : liste des exemples à classifier 
        N.B.: un exemple est un tuple (tenseur, gold class)

        ARGUMENTS DE CLASSE
        self.annotated_examples
        self.not_annotated_examples
        self.k (int) : nombre de clusters pour ce lemme
        self.centroids (list) : liste des centroïdes sous forme d'embeddings
        self.cluster2sense (list) : associe chaque cluster (indice de la liste)
                                    à un sens
        '''

        self.annotated_examples = annotated_examples

        # les gold class ne seront utilisées que pour calculer l'accuracy
        self.not_annotated_examples = not_annotated_examples

        # initialisation de clusters : tous les examples non annotés sont associés 
        # au cluster 0
        self.clusters = np.zeros(len(not_annotated_examples))

        # détermine les sens possibles k (donc le nombre de clusters) 
        # N.B.: dans le cas du constrained Kmeans, cela 
        # peut se faire à l'aide des données annotées, qui représentent tous les sens possibles
        self.k = len(set([example[1] for example in self.annotated_examples]))

        # initialisation des centroïdes : pour chaque sens, le centroïde 
        # correspond à la moyenne des embeddings des exemples annotés
        # Ainsi, chaque centroïde représente un sens
        self.centroids, self.cluster2sense = self.make_centroids(self.annotated_examples)

        # sens le plus fréquent (initialisation à -1)
        self.accuracy_baseline, self.mfs = self.get_accuracy_baseline()


    def make_centroids(self, data):
        '''
        Calcule les coordonnées des centroïdes

        OUTPUT
        centroids (list) : liste des coordonnées des centroïdes dans l'ordre
                           des clusters
        cluster2sense (list) : à chaque indice correpond au numéro d'un sens
        '''

        centroids = []
        # clé : numéro du cluster (ie indice de la liste)
        # valeur : numéro du sens associé
        cluster2sense = []
        # regroupe les sens présents dans le corpus
        senses = set([example[1] for example in data])
        # pour chaque sens
        for sense in senses:
            # on récupère les examples correspondant à ce sens
            examples_sense = [example[0] for example in data if example[1] == sense]
            # le centroide correspond à la moyenne de tous ces examples
            centroids.append(np.mean(examples_sense, axis=0))
            # ajoute le numéro du sens
            cluster2sense.append(sense)

        return centroids, cluster2sense


    def learn_clusters(self):
        '''
        Algorithme d'apprentissage de K-Means : met à jour les coordonnées
        des centroïdes après chaque annotation d'exemple
        '''

        # CALCUL DES DISTANCES ENTRE CHAQUE EXAMPLE ET CHAQUE CENTROIDE
         
        # pour chaque couple (indice, coordonnées) dans les examples
        for i, example in enumerate(self.not_annotated_examples):
            # initialisation de la distance minimum à l'infini
            min_dist = float('inf')
            # pour chaque couple (indice, coordonnées) dans les centroides
            for j, centroid in enumerate(self.centroids):
                # calcul de la distance entre cet example et ce centroide
                d = np.sqrt(np.sum(np.square(centroid - example[0])))
                # si une distance plus faible est trouvée
                if min_dist > d:
                    # la distance ainsi que le centroide sont stockés
                    min_dist = d
                    self.clusters[i] = j
                
            # CALCUL ET MISE A JOUR DES NOUVEAUX CENTROIDES
            
            # donne l'étiquette prédite à cet exemple
            predicted_example = (self.not_annotated_examples[i][0], self.cluster2sense[int(self.clusters[i])])
            # fusionne les données annotées avec cette donnée prédite
            data = self.annotated_examples
            data.append(predicted_example)
            # pour mettre à jour les centroides
            self.make_centroids(data)

    def accuracy(self): 
        '''
        OUTPUT
        - correct (int) : nombre de fois où le sens prédit est le bon
        - len(self.not_annotated_examples) (int): nombre total d'examples non annotés
        
        '''
        correct = 0
        # pour chaque couple (indice, coordonnées) dans les examples
        for i, example in enumerate(self.not_annotated_examples):
            # si l'étiquette prédite est la bonne
            if example[1] == self.cluster2sense[int(self.clusters[i])]:
                # on incrémente correct
                correct += 1
        
        return correct, len(self.not_annotated_examples)
    
    def get_accuracy_baseline(self):
        ''' 
        Retourne le nombre d'étiquettes correspondant au sens le plus fréquent et le sens le plus fréquent.

        OUTPOUT
        accuracy_baseline (float) : l'accuracy correspondante à la baseline
        mfs (int) : sens le plus fréquent
        '''
        
        #si la quantité de donées sélectionnées est suffisante
    
        count_gold = Counter([example[1] for example in self.annotated_examples])
        #accuracy de la baseline : on divise le nombre d'étiquette 
        # dont le gold correspond au sens le plus fréquent par le nombre d'étiquettes prédites
        accuracy_baseline = round(max(count_gold.items(),key = lambda item:item[1])[1]/len(self.annotated_examples),2)
        mfs = max(count_gold.items(),key = lambda item:item[1])[0]

        return accuracy_baseline, mfs


In [139]:
def process_annotated_and_unannotated_sets(instance, data_size):
    '''
    - Dernière étape de la préparation des données pour le clustering.
    INPUT
    instance (str): mot à désambiguiser
    data_size (float): quantité de données à considérer

    OUTPUT
    annotated_set (list): liste des données annotées contenant des tuples(vecteur numpy, sens)
    unannotated_set (list): liste des données (que l'on utilisera comme) non annotées contenant des tuples(vecteur numpy, sens)
    
    '''
    # on définit l'instance
    ext.define_instance(instance, data_size, False)
    # on récupère les données annotées et non annotées
    annotated_set, unannotated_set = ext.get_annotated_and_unannotated_sets()
    annotated_set = [(np.array(embedding),sense) for embedding,sense in annotated_set]
    unannotated_set = [(np.array(embedding),gold) for embedding,gold in unannotated_set]
    
    return annotated_set, unannotated_set

def get_mean_accuracies(instance, data_size, n_repeat, display=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné
    ainsi qu'une moyenne du nombre d'étiquette correctement prédites ainsi que le nombre
    d'étiquettes prédites, nécéssaires au calcul de la micro-average.

    INPUT
    n_repeat (int): nombre de classifications à effectuer
    display (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean_acc (float): moyenne des accuracies calculées
    mean_acc_baseline (float) : moyenne des accuracies de la baseline
    mean_corr_pred (int) : nombre d'étiquettes correctement prédites selon le classifieur
    mean_corr_pred_baseline (int) : nombre d'étiquettes correctement prédites selon la baseline
    n_pred (int) : nombre d'étiquettes prédites
    '''

    # liste de l'accuracy à chaque répétition
    accuracies = []
    
    # liste de l'accuracy de la baseline à chaque répétition
    accuracies_baseline = []
    
    #liste du nombre d'étiquettes correctement prédites
    n_corr_pred = []
    
    #liste du nombre d'étiquettes correctement prédites selon la baseline
    n_corr_pred_baseline = []
    
    
    for n in range(n_repeat) :
            
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données
        ext.define_instance(instance, data_size, False)
        # les données utiles à l'évaluation sont récupérées du classifieur
        # récupération des sets
        annotated_set, unannotated_set = process_annotated_and_unannotated_sets(instance, data_size)

        
        # si les sets ne sont pas vides i.e que la quantité de données sélectionnées est suffisante
        if unannotated_set :
            
            # clustering
            k_Means = Constrained_K_Means(annotated_set, unannotated_set)
            k_Means.learn_clusters()

            # calcul des métriques pour chaque répétition
            correct, num_examples = k_Means.accuracy()
            accuracies.append(correct/num_examples)
            accuracies_baseline.append(k_Means.accuracy_baseline)
            
            n_corr_pred.append(correct)
            n_corr_pred_baseline.append(sum([1 for gold in [gold for (context, gold) in unannotated_set] if k_Means.mfs==gold]))
            
        # lorsque la quantité de données sélectionnées est trop faible
        else :
            accuracies.append(-1.0)
            accuracies_baseline.append(-1.0)
            n_corr_pred.append(-1)
            n_corr_pred_baseline.append(-1)
    
    # calcul des moyennes 
    mean_acc = sum(accuracies)/len(accuracies)
    mean_acc_baseline = sum(accuracies_baseline)/len(accuracies_baseline)
    mean_corr_pred = sum(n_corr_pred)/len(n_corr_pred)
    mean_corr_pred_baseline = sum(n_corr_pred_baseline)/len(n_corr_pred_baseline)

    # si la trace est souhaitée
    if display :
        print(f'{data_size*100}% des données annotées considérées')
        print("instance :", instance)
        print("accuracies :", accuracies)
        print("accuracy moyenne :", round(mean_acc,2))
        print("accuracies de la baseline :",accuracies_baseline)
        print("accuracy moyenne de la baseline :", round(mean_acc_baseline,2),"\n")

    return round(mean_acc,2), round(mean_acc_baseline,2), round(mean_corr_pred,2), round(mean_corr_pred_baseline,2), num_examples 



def get_accuracies(instances, step, n_repeat, display=True):
    '''
    - Permet d'obtenir des accuracies moyennes par lemme pour une certaine quantité 
      de données considérée.
    - Produit des fichiers csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes + macro_average
    instance2acc_base (dict) : associe à chaque instance sa liste de baselines d'accuracies + macro_average
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    instance2acc = {instance : [] for instance in instances}
    instance2acc_base = {instance : [] for instance in instances}
    
    # listes des métriques d'évaluation calculées pour chaque quantité de données
    macro_average_acc = []
    macro_average_acc_base = []
    micro_average = []
    micro_average_baseline = []
    
    # compteurs pour le nombre d'étiquettes correctement prédites selon le classifieur et selon la baseline
    n_corr_pred = 0
    n_corr_pred_baseline = 0
    
    # nombre d'étiquettes prédites pour la quantité de données entrée
    n_pred_tot = 0
    
    # liste qui permettra d'afficher la quantité de données considérées dans le .csv
    data_sizes =[]
            
    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (100%, 75%, 50%, 25%)
        data_size = 0.75 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de ses accuracies moyennes, selon le classifieur et selon la baseline
            # comptage des étiquettes correctement prédites et du nombre d'étiquettes prédites
            mean_acc, mean_acc_baseline, mean_corr_pred, mean_corr_pred_baseline, num_examples = get_mean_accuracies(instance, data_size, n_repeat, display)
            
            instance2acc[instance].append(mean_acc)
            instance2acc_base[instance].append(mean_acc_baseline)
            n_corr_pred += mean_corr_pred
            n_corr_pred_baseline += mean_corr_pred_baseline
            n_pred_tot += num_examples
        
        # on boucle sur les valeurs obtenues pour obtenir leur macro-average et leur micro_average, selon le classifieur et selon la baseline
        accuracies = [round(accuracies[i],2) for accuracies in instance2acc.values() if accuracies[i] != -1.0]
        macro_average_acc.append(sum(accuracies)/len(accuracies))
        
        accuracies_base = [accuracies_baseline[i] for accuracies_baseline in instance2acc_base.values() if accuracies_baseline[i] != -1.0]
        macro_average_acc_base.append(sum(accuracies_base)/len(accuracies_base))
        
        if n_pred_tot > 0 :
            micro_average.append(n_corr_pred/n_pred_tot)
            micro_average_baseline.append(n_corr_pred_baseline/n_pred_tot)
        else :
            micro_average.append(0)
            micro_average_baseline.append(0)
  
    # ajout d'une clé macro_average pour avoir les moyennes de toutes les accuracies moyennes pour chaque instance
    # pour chaque pas
    instance2acc["macro_average"] = macro_average_acc
    instance2acc_base["macro_average"] = macro_average_acc_base
    
    # ajout d'une clé micro_average pour avoir les micro_averages pour chaque instance
    # pour chaque pas
    instance2acc["micro_average"] = micro_average
    instance2acc_base["micro_average"] = micro_average_baseline
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    instance2acc["data_sizes"] = data_sizes
    instance2acc_base["data_sizes"] = data_sizes 

    # export en csv des résultats
    df1 = pd.DataFrame(instance2acc)
    df1.set_index("data_sizes")
    df1.to_csv(f"results/kmeans_accuracies_par_lemme_{n_repeat}_repet.csv")
    
    df2 = pd.DataFrame(instance2acc_base)
    df2.set_index("data_sizes")
    df2.to_csv(f"results/kmeans_accuracies_baseline_par_lemme_{n_repeat}_repet.csv")
    
    
    return instance2acc,instance2acc_base

In [ ]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5
# nombre d'itérations maximal
max_iter = 10

In [140]:
#récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())
# calcul des precisions
get_accuracies(instances, step, n_repeat)

75.0% des données annotées considérées
instance : aboutir
accuracies : [0.8333333333333334, 0.8333333333333334, 0.5833333333333334, 0.6666666666666666, 0.8333333333333334]
accuracy moyenne : 0.75
accuracies de la baseline : [0.79, 0.82, 0.82, 0.82, 0.79]
accuracy moyenne de la baseline : 0.81 

75.0% des données annotées considérées
instance : investir
accuracies : [0.4166666666666667, 0.3333333333333333, 0.5, 0.5, 0.5]
accuracy moyenne : 0.45
accuracies de la baseline : [0.66, 0.63, 0.66, 0.66, 0.68]
accuracy moyenne de la baseline : 0.66 

75.0% des données annotées considérées
instance : traduire
accuracies : [0.5, 0.75, 0.5833333333333334, 0.75, 0.75]
accuracy moyenne : 0.67
accuracies de la baseline : [0.53, 0.45, 0.5, 0.58, 0.5]
accuracy moyenne de la baseline : 0.51 

75.0% des données annotées considérées
instance : témoigner
accuracies : [0.9166666666666666, 0.75, 0.6666666666666666, 0.75, 0.9166666666666666]
accuracy moyenne : 0.8
accuracies de la baseline : [0.89, 0.89, 0.89

({'aboutir': [0.75, 0.78, 0.86, 0.38],
  'investir': [0.45, 0.54, 0.53, 0.3],
  'traduire': [0.67, 0.62, 0.6, 0.43],
  'témoigner': [0.8, 0.82, 0.81, 0.57],
  'juger': [0.73, 0.56, 0.61, 0.42],
  'justifier': [0.52, 0.7, 0.72, 0.28],
  'viser': [0.98, 0.91, 0.82, 0.59],
  'prononcer': [0.3, 0.31, 0.26, 0.31],
  'accomplir': [0.55, 0.63, 0.59, 0.33],
  'convenir': [0.48, 0.46, 0.45, 0.34],
  'acquérir': [0.55, 0.46, 0.51, 0.28],
  'achever': [0.77, 0.66, 0.7, 0.49],
  'observer': [0.57, 0.53, 0.52, 0.3],
  'adapter': [0.52, 0.46, 0.38, 0.24],
  'admettre': [0.63, 0.57, 0.65, 0.45],
  'entraîner': [0.55, 0.62, 0.61, 0.22],
  'payer': [0.3, 0.42, 0.39, 0.17],
  'respecter': [0.68, 0.75, 0.7, 0.65],
  'affecter': [0.58, 0.61, 0.43, 0.11],
  'demeurer': [0.8, 0.77, 0.72, 0.48],
  'aggraver': [0.7, 0.71, 0.72, 0.7],
  'agir': [0.77, 0.59, 0.57, 0.46],
  'ajouter': [0.57, 0.54, 0.44, 0.27],
  'alimenter': [0.58, 0.59, 0.54, 0.27],
  'coûter': [0.6, 0.5, 0.43, 0.36],
  'relancer': [0.62, 0.59,